In [1]:
import csv
import re
from typing import Pattern

In [89]:
path = "data/_57793TransmissionsSurvivors_2014V1.csv"
path_write = "data/new-_57793TransmissionsSurvivors_2014V1.txt"

PATTERN_SUPP_HTML : Pattern = re.compile(r'<(.*?)>')
REPLACE_NO_SPACE : Pattern= re.compile(r"[.;:'!=\%?,\"()\[\]]")
REPLACE_WITH_SPACE : Pattern = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)|°")

In [183]:
# nettoyer le texte 
def preprocess_clean(reviews):
    reviews = reviews.lower() # mettre le texte en minuscule
    reviews = re.split(r'(\d+)', reviews)
    reviews = ' '.join(reviews)
    
    reviews = re.sub(r'\bagrave\b', 'a', reviews) #replacer agrave par a (!à)
    reviews = re.sub(r'\beacut\b|\begrave\b|\bpreacute\b', 'e', reviews)#

    sub_html = re.findall(PATTERN_SUPP_HTML, reviews) #supprimer les balise html
    for html in sub_html:
        if html:
            reviews = re.sub(r'\s+', ' ', reviews)
            reviews = re.sub(r"\'", ' ', reviews)
            reviews = reviews.replace(html, ' ')
            reviews = reviews.replace('< >', ' ')

    reviews = REPLACE_NO_SPACE.sub(' ', reviews) #supprimer tout les caracteres [.;:!=\%?,\"()\[\]]
    reviews = REPLACE_WITH_SPACE.sub(' ', reviews) #supprimer espace entre balise html au cas où y'en a 
    reviews = reviews.replace('ttt', 'traitement') 
    reviews = re.sub(r"([+])", ' plus ', reviews)
    reviews = re.sub(r"([+])", ' ', reviews)
    reviews = re.sub(r'[0-9]', ' ', reviews)
    #reviews = re.sub(r'\b\w{1,1}\b','', reviews)

    reviews = re.sub(' +', ' ', reviews)#supprimer tout espace de plus qui peut exister à la fin de traitement
    #print(reviews)
    return reviews

In [260]:
PATTERN_ASH : Pattern = re.compile(r'\bash\b|\bagent de service hôtelier\b|\bagent de service hotelier\b')
freq_ash = 0

PATTERN_AIDE_SOIGNANT : Pattern = re.compile(r'\bas\b|\bas\.\b|\baide-soignant\b|\baide soignant\b')
freq_as = 0

PATTERN_IDE : Pattern = re.compile(r'\bide\b')
freq_ide = 0

PATTERN_IDEC : Pattern = re.compile(r'\bchange\b')
freq_idec = 0

# /^((?!\bmedecin-co\b).)*$/|/^((?!\bmedco\b).)*$/| .*médecin(?=creek).*|
PATTERN_MEDECIN : Pattern = re.compile(r'\bmt\b|\bmt\.\b|\bdr\b|\bdr\.\b|\b(m(é|e)decin)?(?!-co)\b|\bdocteur\b')
freq_medecin = 0

In [276]:
PATTERN_GRIPPE : Pattern = re.compile(r'(\bgri(pp|p)e\b)')
patrn_GRIPPE = re.compile(r'(vaccin(|ation) (anti|contre))')

freq_GRIPPE = 0

In [277]:
#file_txt = open(path_write, "a")

In [278]:
with open(path, 'r') as csvfile:
    for id_resident, id_etablissement, date_T, descrip in  csv.reader(csvfile, delimiter=';'):
        descrip = preprocess_clean(descrip)

        _ash = PATTERN_ASH.findall(descrip)
        if _ash :
            #print(_ash)
            freq_ash += len(_ash)

        _as = PATTERN_AIDE_SOIGNANT.findall(descrip)
        if _as :
            #print(_as)
            freq_as += len(_as)

        _ide = PATTERN_IDE.findall(descrip)
        if _ide :
            freq_ide += len(_ide)

        _idec = PATTERN_IDEC.findall(descrip)
        if _idec :
            # print(descrip)
            freq_idec += len(_idec)

        _medecin = PATTERN_MEDECIN.findall(descrip)
        # print(_medecin)
        for x, y in _medecin:
            if len(x) :
                # print(x)
                freq_medecin += 1
        _grippe = PATTERN_GRIPPE.findall(descrip)

        
        if _grippe :
            relo = patrn_GRIPPE.findall(descrip)
            if relo:
                #relo = print(''.join(relo[0]))
                relo = descrip.split(relo[0][0])
                #print(relo)  
                
            if len(relo) < 1 :
                print(id_resident, descrip)
                freq_GRIPPE += len(_grippe)

        #file_txt.write(descrip + '\n')
        # print(descrip)
        # break
#file_txt.close()

2321 a eu son vaccin de la grippe ce jour a surveiller
15767 revu avec mr vitureau refuse toujours le vaccin de la grippe
15767 vaccin grippe vaxigrip ser ml fait le à renouveler le n lot l prise en charge commentaire apyrétique bras droit
17126 vaccination grippe
23277 fille mme padovani contacté pour le formulaire anti grippe message laissé sur le repondeur 
24104 son medecin traitant a refusé de prescrire un vaccin pour la grippe tout en sachant que nous n arrivions pas à acquérir la prise en charge de la sécurité sociale 
25109 reclamait un traitement homéopatique pour prevenir de la grippe
26371 évertion de la paupiére sup droite avec irritationcat biocidance matin refuse violemment la vaccination grippe
27671 vacin grippe fait ce jouradministré en sous cutimmugriplot mx exp t 
28309 elle a fait une reaction à vaccine de la gripe rougeur plus pruride de la glace mis 
28383 visite de son medecin elle a une grippe intestinale vu avec irena qui etait encore presente lors du passage d

In [258]:
print('Agent de service hotelier : ' + str(freq_ash))
print('Aide soignant : ' + str(freq_as))
print('IDE : ' + str(freq_ide))
print('IDEC : ' + str(freq_idec))
print('Medecin : ' + str(freq_medecin))
print('Grippe : ' + str(freq_GRIPPE))

Agent de service hotelier : 45
Aide soignant : 589
IDE : 3192
IDEC : 1184
Medecin : 1916
Grippe : 34


In [238]:
s = "programme : 05/12/2014 au 05/12/2015objectifs : 05/12/2014, poirier morgane (psychomotricienne)16h46 :\
se sentir en sécurité face à son incontinence nocturneactions : 05/12/2014, poirier morgane (psychomotricienne)\
16h46 :  a la demande de la résidente ajouter une anaform en plus dans la pull-upsressources : 05/12/2014, \
poirier morgane (psychomotricienne)16h46 :  responsable : soignants, gouvernante, idecindicateur : 1/nuitsuivi\
: plan de soinévaluation : 05/12/2014, poirier morgane (psychomotricienne)16h46 :  selon le souhait \
de la résidente, accordé par l'idec et la gouvernante. la vaccin contre grippe "

s = preprocess_clean(s)
print(s)


    

programme au objectifs poirier morgane psychomotricienne h se sentir en sécurité face à son incontinence nocturneactions poirier morgane psychomotricienne h a la demande de la résidente ajouter une anaform en plus dans la pull upsressources poirier morgane psychomotricienne h responsable soignants gouvernante idecindicateur nuitsuivi plan de soinévaluation poirier morgane psychomotricienne h selon le souhait de la résidente accordé par l idec et la gouvernante la vaccin contre grippe 
['programme au objectifs poirier morgane psychomotricienne h se sentir en sécurité face à son incontinence nocturneactions poirier morgane psychomotricienne h a la demande de la résidente ajouter une anaform en plus dans la pull upsressources poirier morgane psychomotricienne h responsable soignants gouvernante idecindicateur nuitsuivi plan de soinévaluation poirier morgane psychomotricienne h selon le souhait de la résidente accordé par l idec et la gouvernante la ', ' grippe ']
